In [2]:
#Import libraries and settings

#To perform array operations
import numpy as np 

#Main plotting library
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator, FormatStrFormatter)

#Library for accessing files in the directory
import os

#To create map projections for plots
from cartopy import crs as ccrs

#To add Basemap for mapping features
os.environ['PROJ_LIB'] = 'C:\\AKH\\Programming\\basemap-master\\lib\\mpl_toolkits\\basemap\\data'
from mpl_toolkits.basemap import Basemap

#To read in netCDF files
from netCDF4 import Dataset

#Library for using math functions
import math

#Library for collecting lists of files from folders
import glob

#Library for working with tar files
import tarfile

#Library for creating movies (animations)
import imageio

import warnings
warnings.filterwarnings("ignore")

#Sets font size to 12
plt.rcParams.update({'font.size': 12})

#Option to keep numpy from printing in scientific notation by default
np.set_printoptions(suppress=True)

In [ ]:
#Work with a single file (MODIFY path, file name)

#Enter file name
fname = 'D://Data/20191025/ABI ADP/' + 'OR_ABI-L2-ADPC-M6_G17_s20192982001196_e20192982003569_c20192982004415.nc'

#Set the file name to read
file_id = Dataset(fname)

#Check the contents of the entire file
##print(file_id)

#Check the "Variables" metadata
##print(file_id.variables)

#Check the "Smoke" metadata
##print(file_id.variables['Smoke'])

#Check the "Dust" metadata
##print(file_id.variables['Dust'])

#Check the "DQF" metadata
##print(file_id.variables['DQF'])

In [ ]:
#Check the datatype for the variables of interest

print('Smoke data type is', (file_id.variables['Smoke'][:,:].dtype))
print('Dust data type is', (file_id.variables['Dust'][:,:].dtype))
print('DQF data type is', (file_id.variables['DQF'][:,:].dtype))

In [19]:
#Select and process data from a single file

#Select all of the smoke pixels using the "Smoke" variable
#smoke absent = 0, smoke present = 1
Smoke = file_id.variables['Smoke'][:,:]
Smoke_Mask = (Smoke == 0)
Smoke_Present = np.ma.masked_where(Smoke_Mask, Smoke)

#Select all of the good quality smoke pixels using the "DQF" variable (bit 0)
#good = 0, invalid = 1
DQF = file_id.variables['DQF'][:,:]
Smoke_Quality = ((DQF & 0) == 1)
Smoke_Good = np.ma.masked_where(Smoke_Quality, Smoke_Present)

#Select all of the pixels within valid SZA using the "DQF" variable (bit 7)
#within valid SZA for smoke and dust = 0, outside of valid SZA for smoke and dust = 128
SZA_Mask = ((DQF & 128) == 128)
Smoke_All = np.ma.masked_where(SZA_Mask, Smoke_Good)

#Subset the good quality, valid SZA smoke pixels into high-, medium-, and low-confidence using the "DQF" variable (bits 2-3)
#high-confidence = 12, medium-confidence = 4, low-confidence = 0
HC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 12)
Smoke_HC = np.ma.masked_where(HC_Smoke_Mask, Smoke_All)
Smoke_High = Smoke_HC + 2
MC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 4)
Smoke_MC = np.ma.masked_where(MC_Smoke_Mask, Smoke_All)
Smoke_Medium = Smoke_MC + 1
LC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 0)
Smoke_LC = np.ma.masked_where(LC_Smoke_Mask, Smoke_All)
Smoke_Low = Smoke_LC
Smoke_Total = Smoke_High.filled(0) + Smoke_Medium.filled(0) + Smoke_Low.filled(0)

#Select all of the dust pixels using "Dust" variable
#dust absent = 0, dust present = 1
Dust = file_id.variables['Dust'][:,:]
Dust_Mask = (Dust == 0)
Dust_Present = np.ma.masked_where(Dust_Mask, Dust)

#Select all of the good quality dust pixels using the "DQF" variable (bit 1)
#good = 0, invalid = 2
Dust_Quality = ((DQF & 2) == 2)
Dust_Good = np.ma.masked_where(Dust_Quality, Dust_Present)

#Select all of the dust pixels outside of sun-glint using the "DQF" variable (bit 6)
#outside sun-glint = 0, within sun-glint = 64
Glint_Mask = ((DQF & 64) == 64)
Dust_Glint = np.ma.masked_where(Glint_Mask, Dust_Good)

#Select all of the pixels within valid SZA using the "DQF" variable (bit 7)
Dust_All = np.ma.masked_where(SZA_Mask, Dust_Glint)

#Subset the good quality dust pixels into high-, medium-, and low-confidence pixels using the "DQF" variable (bits 4-5)
#high-confidence = 48, medium-confidence = 16, low-confidence = 0
HC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 48)
Dust_HC = np.ma.masked_where(HC_Dust_Mask, Dust_All)
Dust_High = Dust_HC + 2
MC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 16)
Dust_MC = np.ma.masked_where(MC_Dust_Mask, Dust_All)
Dust_Medium = Dust_MC + 1
LC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 0)
Dust_LC = np.ma.masked_where(LC_Dust_Mask, Dust_All)
Dust_Low = Dust_LC
Dust_Total = Dust_High.filled(0) + Dust_Medium.filled(0) + Dust_Low.filled(0)

# Algorithm to convert lat/long radian values to lat/long in degrees
##################################################################################################################

proj_info = file_id.variables['goes_imager_projection']
lon_origin = proj_info.longitude_of_projection_origin
H = proj_info.perspective_point_height+proj_info.semi_major_axis
r_eq = proj_info.semi_major_axis
r_pol = proj_info.semi_minor_axis

# Data info
lat_rad_1d = file_id.variables['x'][:]
lon_rad_1d = file_id.variables['y'][:]
        
# Create meshgrid filled with radian angles
lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

# lat/lon calculus routine from satellite radian angle vectors
lambda_0 = (lon_origin*np.pi)/180.0

a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
c_var = (H**2.0)-(r_eq**2.0)

r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
s_y = - r_s*np.sin(lat_rad)
s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

Lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
Lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
     
##################################################################################################################

In [ ]:
#Review processed data (sanity check)

print('Smoke_Total maximum is', np.max(Smoke_Total), '; Smoke_Total mimimum is', np.min(Smoke_Total))
print('Dust_Total maximum is', np.max(Dust_Total), '; Dust_Total mimimum is', np.min(Dust_Total))

In [ ]:
#Plot aerosol detection (smoke and dust) from a single data file

#Set up figure and map projection
fig = plt.figure(figsize = (8,4))
ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())

#Set coastlines and national/state boundaries at low ("l") resolution using Basemap
m1 = Basemap(resolution = 'l')
m1.drawcoastlines(color = 'black')
m1.drawcountries(color = 'black')
m1.drawstates(color = 'white')
m1.drawlsmask(land_color = 'grey', ocean_color = 'lightgrey', lakes = True)

#Add and format title (MODIFY name)
plt.title('GOES-17 ABI \n Aerosol Detection \n 20:01 UTC, 25 Oct 2019', y = 1.05, size = 15, weight = 'bold')

#Zoom to coordinates
plt.axis([-128,-64,20,52])

#Create contour plot of smoke and dust data
Plot1 = ax.contourf(Lon, Lat, Smoke_All, levels = [0.5,1.5,2.5,3.5], colors = ['hotpink', 'mediumvioletred', 'purple'])
Plot2 = ax.contourf(Lon, Lat, Dust_All, levels = [0.5,1.5,2.5,3.5], colors =['yellow', 'sandybrown', 'sienna' ])

#Add and format colorbars
cbar1_ax = fig.add_axes([0.175, 0, 0.3, 0.05])
cbar2_ax = fig.add_axes([0.55, 0, 0.3, 0.05])

cb1 = plt.colorbar(Plot1, cax=cbar1_ax, orientation = 'horizontal', pad = 0.1)
cb1.set_label(label = 'Smoke Confidence', size = 'large', weight = 'bold')
cb1.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb1.ax.xaxis.set_minor_locator(AutoMinorLocator(2))
cb1.ax.set_xticklabels(['Low', 'Med', 'High'], minor = True, horizontalalignment='center')
cb1.ax.tick_params(which = 'both', length=0, labelsize = 'medium')

cb2 = plt.colorbar(Plot2, cax=cbar2_ax, orientation = 'horizontal', pad = 0.1)
cb2.set_label(label = 'Dust Confidence', size = 'large', weight = 'bold')
cb2.ax.xaxis.set_major_formatter(ticker.NullFormatter())
cb2.ax.xaxis.set_minor_locator(AutoMinorLocator(2))
cb2.ax.set_xticklabels(['Low', 'Med', 'High'], minor = True, horizontalalignment='center')
cb2.ax.tick_params(which = 'both', length=0, labelsize = 'medium')

#Show figure
plt.show()

#Save figure (MODIFY path, file name)
fig.savefig('D://Data/20191025/ABI ADP/Figures/G17_ABI-ADP_20191025_2001', bbox_inches = 'tight', dpi=150)

In [ ]:
#Make multiple individual figures of aerosol detection (one plot from each data file)

#Collect all of the .nc files in given subdirectory (MODIFY path)
file_list = sorted(glob.glob('D://Data/20191025/ABI ADP/' + '*.nc'))

#Loop through data files, making/saving multiple figures, one for each data file
for x in file_list:
    file_id = Dataset(x)  
    
    #Set up figure and map projection
    fig = plt.figure(figsize = (8,4))
    ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())

    #Set coastlines and national/state boundaries at low ("l") resolution using Basemap
    m1 = Basemap(resolution = 'l')
    m1.drawcoastlines(color = 'black')
    m1.drawcountries(color = 'black')
    m1.drawstates(color = 'white')
    m1.drawlsmask(land_color = 'grey', ocean_color = 'lightgrey', lakes = True)

    #Add and format title (MODIFY name)
    plt.title('GOES-17 ABI \n Aerosol Detection \n' + x[56:58] + ':' + x[58:60] + ' UTC, 25 Oct 2019', y = 1.05, size = 15, weight = 'bold')

    #Zoom to coordinates
    plt.axis([-128,-64,20,52])
    
    #Select all of the smoke pixels using the "Smoke" variable
    #smoke absent = 0, smoke present = 1
    Smoke = file_id.variables['Smoke'][:,:]
    Smoke_Mask = (Smoke == 0)
    Smoke_Present = np.ma.masked_where(Smoke_Mask, Smoke)
    
    #Select all of the good quality smoke pixels using the "DQF" variable (bit 0)
    #good = 0, invalid = 1
    DQF = file_id.variables['DQF'][:,:]
    Smoke_Quality = ((DQF & 0) == 1)
    Smoke_Good = np.ma.masked_where(Smoke_Quality, Smoke_Present)
    
    #Select all of the pixels within valid SZA using the "DQF" variable (bit 7)
    #within valid SZA for smoke and dust = 0, outside of valid SZA for smoke and dust = 128
    SZA_Mask = ((DQF & 128) == 128)
    Smoke_All = np.ma.masked_where(SZA_Mask, Smoke_Good)
    
    #Subset the good quality, valid SZA smoke pixels into high-, medium-, and low-confidence using the "DQF" variable (bits 2-3)
    #high-confidence = 12, medium-confidence = 4, low-confidence = 0
    HC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 12)
    Smoke_HC = np.ma.masked_where(HC_Smoke_Mask, Smoke_All)
    Smoke_High = Smoke_HC + 2
    MC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 4)
    Smoke_MC = np.ma.masked_where(MC_Smoke_Mask, Smoke_All)
    Smoke_Medium = Smoke_MC + 1
    LC_Smoke_Mask = ((DQF & 4) & (DQF & 8) != 0)
    Smoke_LC = np.ma.masked_where(LC_Smoke_Mask, Smoke_All)
    Smoke_Low = Smoke_LC
    Smoke_Total = Smoke_High.filled(0) + Smoke_Medium.filled(0) + Smoke_Low.filled(0)
    
    #Select all of the dust pixels using "Dust" variable
    #dust absent = 0, dust present = 1
    Dust = file_id.variables['Dust'][:,:]
    Dust_Mask = (Dust == 0)
    Dust_Present = np.ma.masked_where(Dust_Mask, Dust)
    
    #Select all of the good quality dust pixels using the "DQF" variable (bit 1)
    #good = 0, invalid = 2
    Dust_Quality = ((DQF & 2) == 2)
    Dust_Good = np.ma.masked_where(Dust_Quality, Dust_Present)
    
    #Select all of the dust pixels outside of sun-glint using the "DQF" variable (bit 6)
    #outside sun-glint = 0, within sun-glint = 64
    Glint_Mask = ((DQF & 64) == 64)
    Dust_Glint = np.ma.masked_where(Glint_Mask, Dust_Good)
    
    #Select all of the pixels within valid SZA using the "DQF" variable (bit 7)
    Dust_All = np.ma.masked_where(SZA_Mask, Dust_Glint)
    
    #Subset the good quality dust pixels into high-, medium-, and low-confidence pixels using the "DQF" variable (bits 4-5)
    #high-confidence = 48, medium-confidence = 16, low-confidence = 0
    HC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 48)
    Dust_HC = np.ma.masked_where(HC_Dust_Mask, Dust_All)
    Dust_High = Dust_HC + 2
    MC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 16)
    Dust_MC = np.ma.masked_where(MC_Dust_Mask, Dust_All)
    Dust_Medium = Dust_MC + 1
    LC_Dust_Mask = ((DQF & 16) & (DQF & 32) != 0)
    Dust_LC = np.ma.masked_where(LC_Dust_Mask, Dust_All)
    Dust_Low = Dust_LC
    Dust_Total = Dust_High.filled(0) + Dust_Medium.filled(0) + Dust_Low.filled(0)
    
    # Algorithm to convert lat/long radian values to lat/long in degrees
    ##################################################################################################################
    
    proj_info = file_id.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis
    
    # Data info
    lat_rad_1d = file_id.variables['x'][:]
    lon_rad_1d = file_id.variables['y'][:]
    
    # Create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)
    
    # lat/lon calculus routine from satellite radian angle vectors
    lambda_0 = (lon_origin*np.pi)/180.0
    
    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)
    
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    
    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)
    
    Lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    Lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    ##################################################################################################################
    
    #Create contour plots of smoke and dust data
    Plot1 = ax.contourf(Lon, Lat, Smoke_Total, levels = [0.5,1.5,2.5,3.5], colors = ['hotpink', 'mediumvioletred', 'purple'])
    Plot2 = ax.contourf(Lon, Lat, Dust_Total, levels = [0.5,1.5,2.5,3.5], colors =['yellow', 'sandybrown', 'sienna' ])
    
    #Add and format colorbars
    cbar1_ax = fig.add_axes([0.175, 0, 0.3, 0.05])
    cbar2_ax = fig.add_axes([0.55, 0, 0.3, 0.05])
    
    cb1 = plt.colorbar(Plot1, cax=cbar1_ax, orientation = 'horizontal', pad = 0.1)
    cb1.set_label(label = 'Smoke Confidence', size = 'large', weight = 'bold')
    cb1.ax.xaxis.set_major_formatter(ticker.NullFormatter())
    cb1.ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    cb1.ax.set_xticklabels(['Low', 'Med', 'High'], minor = True, horizontalalignment='center')
    cb1.ax.tick_params(which = 'both', length=0, labelsize = 'medium')
    
    cb2 = plt.colorbar(Plot2, cax=cbar2_ax, orientation = 'horizontal', pad = 0.1)
    cb2.set_label(label = 'Dust Confidence', size = 'large', weight = 'bold')
    cb2.ax.xaxis.set_major_formatter(ticker.NullFormatter())
    cb2.ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    cb2.ax.set_xticklabels(['Low', 'Med', 'High'], minor = True, horizontalalignment='center')
    cb2.ax.tick_params(which = 'both', length=0, labelsize = 'medium')
    
    #Show figure
    plt.show()
    
    #Save figure (MODIFY path, filename)
    filename = 'G17_ABI_ADP_20191025_' + x[56:60]
    fig.savefig('D://Data/20191025/ABI ADP/Figures/' + filename, bbox_inches = 'tight', dpi = 150)
    
    #Erase plot so next one can be generated
    plt.close()

In [ ]:
#Make an animation (movie) of multiple smoke/dust figures

#Collect all of the graphics files (figures) in given subdirectory (MODIFY path)
file_list = sorted(glob.glob('D://Data/20191025/ABI ADP/Figures/' + '*.png'))

#Create an empty list to store figures
images = []

#Set duration of each frame in animation (in seconds)
duration = 1

#Loop through graphics files and append to animation
for x in file_list:
    images.append(imageio.imread(x))

#Save final animation (MODIFY path, filename)
imageio.mimsave('D://Data/20191025/ABI ADP/Figures/ADP-Animation.gif', images, 'GIF', duration=duration)

print('Done')